In [15]:
import os
from huggingface_hub import InferenceClient

client = InferenceClient(
    provider="hf-inference",
    api_key=os.environ["HF_TOKEN"],
)
model_name = "NlpHUST/ner-vietnamese-electra-base"
query = "Mục lục NỘI THIÊN NỘI THIÊN 2 NỘI THIÊN 3 Các sách chú giải Trang Tử HỌC THUYẾT CỦA TRANG TỬ HỌC THUYẾT CỦA TRANG TỬ 2 TIÊU- DIÊU- DU TỔNG BÌNH TỀ- VẬT- LUẬN Phải Quấy và Xấu Tốt ĐỨC SUNG PHÙ"


result = client.token_classification(
    query,
    model=model_name,
)

result

/home/octoopt/workspace/projects/learn-from-basics/nlp-vietnamese-phd/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[]

In [ ]:
from underthesea import word_tokenize

tokens = word_tokenize(query)
print(tokens)

for token in tokens:
    print(token)
    result = client.token_classification(model=model_name, text=token)
    print(result)

In [ ]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification
# from transformers import pipeline

# tokenizer = AutoTokenizer.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
# model = AutoModelForTokenClassification.from_pretrained("NlpHUST/ner-vietnamese-electra-base")

# nlp = pipeline("ner", model=model, tokenizer=tokenizer)
# example = "Liên quan vụ việc CSGT bị tố đánh dân, trúng một cháu nhỏ đang ngủ, đang lan truyền trên mạng xã hội, Đại tá Nguyễn Văn Tảo, Phó Giám đốc Công an tỉnh Tiền Giang vừa có cuộc họp cùng Chỉ huy Công an huyện Châu Thành và một số đơn vị nghiệp vụ cấp tỉnh để chỉ đạo làm rõ thông tin."

# ner_results = nlp(example)
# print(ner_results)
# model_name = "undertheseanlp/vietnamese-ner-v1.4.0a2"

[]

In [5]:
import os
from openai import AsyncOpenAI
from pydantic import BaseModel


class NERResponse(BaseModel):
    end: int
    score: float
    start: int
    word: str
    entity: str
    entity_group: str


NER_PROMPT = """
Nhiệm vụ: Nhận diện thực thể có tên (NER) trong văn bản tiếng Việt.

Bạn là mô hình NER. Hãy trích xuất các thực thể trong văn bản và phân loại chúng vào một trong các nhóm sau:
- PER: Tên người
- LOC: Địa danh
- ORG: Tổ chức
- TME: Thời gian
- TITLE: Tựa đề
- NUM: Số

Trả về kết quả dưới dạng danh sách JSON. Mỗi thực thể gồm các trường:
- start: vị trí bắt đầu (ký tự)
- end: vị trí kết thúc (ký tự)
- word: văn bản của thực thể
- entity_group: loại thực thể

Chỉ trả về JSON. Nếu không có thực thể, trả về `[]`.
"""


client = AsyncOpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    # base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    base_url="https://api.groq.com/openai/v1/",
)

query = "Mục lục NỘI THIÊN NỘI THIÊN 2 NỘI THIÊN 3 Các sách chú giải Trang Tử HỌC THUYẾT CỦA TRANG TỬ HỌC THUYẾT CỦA TRANG TỬ 2 TIÊU- DIÊU- DU TỔNG BÌNH TỀ- VẬT- LUẬN Phải Quấy và Xấu Tốt ĐỨC SUNG PHÙ"

completion = await client.chat.completions.create(
    model="gemma2-9b-it",
    messages=[
        {"role": "system", "content": NER_PROMPT},
        {"role": "user", "content": query},
    ],
    # response_format=NERResponse,
)


print(completion.choices[0].message.content)

[
  {
    "start": 1,
    "end": 5,
    "word": "TRANG",
    "entity_group": "PER"
  },
  {
    "start": 20,
    "end": 37,
    "word": "TIÊU-DIÊU-DU TỔNG BÌNH",
    "entity_group": "TITLE"
  },
  {
    "start": 41,
    "end": 50,
    "word": "TỀ-VẬT-LUẬN",
    "entity_group": "TITLE"
  }
]



In [4]:
import json
import re

# Your string with markdown code blocks
markdown_string = completion.choices[0].message.content

# Remove markdown code block markers
json_string = re.sub(r"^```json\n|\n```$", "", markdown_string.strip())

# Parse the JSON
try:
    entities = json.loads(json_string)
    print("Successfully parsed JSON:")
    for entity in entities:
        # print(f"Entity: {entity['word']} (Type: {entity['entity_group']})")
        print(entity)
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")

Successfully parsed JSON:
{'start': 2, 'end': 5, 'word': 'TRANG', 'entity_group': 'PER', 'score': 0.99}
{'start': 8, 'end': 13, 'word': 'TÙY', 'entity_group': 'PER', 'score': 0.99}
{'start': 29, 'end': 38, 'word': 'TRANG TỬ', 'entity_group': 'PER', 'score': 0.99}


In [3]:
models = await client.models.list()
for model in models.data:
    print(model.id)

meta-llama/llama-4-maverick-17b-128e-instruct
meta-llama/llama-guard-4-12b
deepseek-r1-distill-llama-70b
llama-3.1-8b-instant
compound-beta
playai-tts
compound-beta-mini
llama3-70b-8192
qwen/qwen3-32b
whisper-large-v3
llama3-8b-8192
meta-llama/llama-4-scout-17b-16e-instruct
distil-whisper-large-v3-en
qwen-qwq-32b
gemma2-9b-it
allam-2-7b
whisper-large-v3-turbo
mistral-saba-24b
meta-llama/llama-prompt-guard-2-22m
playai-tts-arabic
llama-3.3-70b-versatile
meta-llama/llama-prompt-guard-2-86m


In [7]:
from underthesea import ner

query = "Sách Trang tử, theo Hán- thư Nghệ- Văn- Chí, thì có đến năm mươi hai(52) thiên. Nay chỉ thấy còn có ba mươi ba (33) thiên."

result = ner(query, deep=True)
result 

[{'entity': 'B-PER',
  'score': np.float32(0.9695143),
  'index': 10,
  'word': 'Nghệ',
  'start': 29,
  'end': 33},
 {'entity': 'B-PER',
  'score': np.float32(0.998722),
  'index': 12,
  'word': 'Văn',
  'start': 35,
  'end': 38},
 {'entity': 'B-PER',
  'score': np.float32(0.9946812),
  'index': 14,
  'word': 'Chí',
  'start': 40,
  'end': 43}]

In [3]:
base_entities = [
    "PER",
    "ORG",
    "LOC",
    "ORG",
    "TME",
    "TITLE",
    "NUM",
]

for r in result: 
    base_entity = r.get('entity').split('-')[-1]
    if base_entity in base_entities:    
        print(r)



{'entity': 'B-PER', 'score': np.float32(0.99410796), 'index': 4, 'word': 'NỘI', 'start': 8, 'end': 11}
{'entity': 'I-PER', 'score': np.float32(0.9984523), 'index': 7, 'word': 'THIÊN', 'start': 12, 'end': 17}
{'entity': 'B-PER', 'score': np.float32(0.62662), 'index': 11, 'word': 'NỘI', 'start': 18, 'end': 21}
{'entity': 'I-PER', 'score': np.float32(0.9986203), 'index': 14, 'word': 'THIÊNỘI', 'start': 22, 'end': 33}
{'entity': 'I-PER', 'score': np.float32(0.9710959), 'index': 22, 'word': 'THIÊNỌC', 'start': 34, 'end': 72}
{'entity': 'I-PER', 'score': np.float32(0.71561724), 'index': 36, 'word': 'THUY', 'start': 73, 'end': 77}
{'entity': 'B-PER', 'score': np.float32(0.99817026), 'index': 45, 'word': 'TRANG', 'start': 84, 'end': 89}
{'entity': 'I-PER', 'score': np.float32(0.9992786), 'index': 48, 'word': 'TỬỌ', 'start': 90, 'end': 95}
{'entity': 'I-PER', 'score': np.float32(0.58109605), 'index': 53, 'word': 'THUY', 'start': 97, 'end': 101}
{'entity': 'B-PER', 'score': np.float32(0.99750465

In [6]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/ner-vietnamese-electra-base")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "Sách Trang tử, theo Hán- thư Nghệ- Văn- Chí, thì có đến năm mươi hai(52) thiên. Nay chỉ thấy còn có ba mươi ba (33) thiên."

ner_results = nlp(example)
print(ner_results)


Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[]


In [9]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("nguyendangsonlam/lsg-ner-vietnamese-electra-base-1024")
model = AutoModelForTokenClassification.from_pretrained("nguyendangsonlam/lsg-ner-vietnamese-electra-base-1024")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "Sách Trang tử, theo Hán- thư Nghệ- Văn- Chí, thì có đến năm mươi hai(52) thiên. Nay chỉ thấy còn có ba mươi ba (33) thiên."

ner_results = nlp(example)
print(ner_results)


Device set to use cuda:0


[]
